In [1]:
from google.colab import files
uploaded = files.upload()

Saving AB_NYC_2019.csv to AB_NYC_2019.csv


In [2]:
import pandas as pd
import numpy as np

# Read CSV
df = pd.read_csv("AB_NYC_2019.csv")

# CHECK:
print(df.shape)
df.head()

(48895, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
!pip install category_encoders

     |████████████████████████████████| 81kB 4.3MB/s 


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import pandas as pd
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [16]:
def accum(neighbourhood_group, neighbourhood, latitude,	longitude, room_type,	minimum_nights,	number_of_reviews,	calculated_host_listings_count,	availability_365, df):
    data = {"neighbourhood_group": neighbourhood_group,
            "neighbourhood": neighbourhood,
            "latitude": latitude,
            "longitude": longitude,
            "room_type": room_type,
            "minimum_nights": minimum_nights,
            "number_of_reviews": number_of_reviews,
            "calculated_host_listings_count": calculated_host_listings_count,
            "availability_365": availability_365}
    
    # Info DataFrame
    info = pd.DataFrame(data, index=[0])

    # Append
    new_df = pd.concat([df, info], axis=0)
    
    return new_df

info = accum("Staten Island", "Port Richmond", 40.615542, -74.14331, "Private room", 10, 0, 0, 60, df)
info.tail()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
48891,36485057.0,Affordable room in Bushwick/East Williamsburg,6570630.0,Marisol,Brooklyn,Bushwick,40.701840,-73.93317,Private room,40.0,4,0,NaN,NaN,2,36
48892,36485431.0,Sunny Studio at Historical Neighborhood,23492952.0,Ilgar & Aysel,Manhattan,Harlem,40.814750,-73.94867,Entire home/apt,115.0,10,0,NaN,NaN,1,27
48893,36485609.0,43rd St. Time Square-cozy single bed,30985759.0,Taz,Manhattan,Hell's Kitchen,40.757510,-73.99112,Shared room,55.0,1,0,NaN,NaN,6,2
48894,36487245.0,Trendy duplex in the very heart of Hell's Kitchen,68119814.0,Christophe,Manhattan,Hell's Kitchen,40.764040,-73.98933,Private room,90.0,7,0,NaN,NaN,1,23
0,NaN,NaN,NaN,NaN,Staten Island,Port Richmond,40.615542,-74.14331,Private room,NaN,10,0,NaN,NaN,0,60


In [17]:
import warnings
warnings.filterwarnings('ignore')

def preprocessing(df):
  """
  Preprocesses the data.

  Input: DataFrame

  Output: X_train, X_test, y_train, y_test
  """
  # Copying DF
  dfx = df.copy()

  ## EDA
  # Dropping Columns
  dfx.drop(columns=["host_name", "last_review", "reviews_per_month"], inplace = True)

  # Removing -- Custom Outliers
  dfx = dfx[(dfx["price"] > 0) & 
            (dfx["price"] < 10000)]
  
  # New Column -- 'log_price'
  dfx["log_price"] = np.log(dfx["price"].values)


  # Target and Features
  target = "log_price"
  features = ["neighbourhood_group",
              "neighbourhood",
              "latitude",
              "longitude",
              "room_type",
              "minimum_nights",
              "number_of_reviews",
              "calculated_host_listings_count",
              "availability_365"]
  
  # X Features Matrix
  X = dfx[features]

  # y taregt vector
  y = dfx[target]
  
  # Mapping - 'room_type'
  room_type_dict = {"Shared room":1, "Private room":2, "Entire home/apt":3}
  X["room_type"] = X["room_type"].map(room_type_dict)
  # print(X["room_type"])

  # Train Test Split
  X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=.2,
                                                      random_state=42)

  # Preprocess Pipeline -- OrdinalEncoder and StandardScaler
  preprocess = make_pipeline(
      ce.OrdinalEncoder(),
      StandardScaler()
  )

  # Fit Transform and Transform Training and Testing Data
  X_train = preprocess.fit_transform(X_train)
  X_test = preprocess.transform(X_test)

  # Create DataFrame for X Matrices
  X_train_df = pd.DataFrame(X_train, columns=features) 
  X_test_df = pd.DataFrame(X_test, columns=features)
  print(X_train_df.shape,
        X_test_df.shape, 
        X_train.shape, 
        X_test.shape, 
        y_train.shape, 
        y_test.shape)

  # Return
  return X_train_df, X_test_df, X_train, X_test, y_train, y_test


# Calling Function
X_train_df, X_test_df, X_train, X_test, y_train, y_test = preprocessing(info)


# # CHECK:
# X_train_df.head()

(39104, 9) (9777, 9) (39104, 9) (9777, 9) (39104,) (9777,)


In [7]:
# neighbourhood_group = "Staten Island" 
#     neighbourhood = "Port Richmond"
#     latitude = 40.615542
#     longitude = -74.14331
#     room_type = "Private room"
#     minimum_nights = 10
#     number_of_reviews = 0
#     calculated_host_listing_count = 0
#     availability_365 = 60

In [25]:
from sklearn.ensemble import RandomForestRegressor

def rfr_function(X_train, y_train):
  
  # Instantiate Model
  rfr = RandomForestRegressor()
  
  # Fit
  rfr.fit(X_train[:-1], y_train[:-1])
  
  # Training Prediction
  train_pred = rfr.predict([X_train[-1]])

  return np.exp(train_pred)

prediction = rfr_function(X_train, y_train)


In [26]:
prediction

array([93.01235648])

In [ ]:
# Defining Function
def df_maker(y_vector, y_pred_vector):
  
  # Training Data
  log_price = pd.DataFrame(y_vector, columns=["log_price"])
  log_price_pred = pd.DataFrame(y_pred_vector, columns=["log_price_pred"])

  unlog_price = np.exp(log_price.values)
  unlog_price = pd.DataFrame(unlog_price, columns=["unlog_price"])

  unlog_pred_price = np.exp(log_price_pred.values)
  unlog_pred_price = pd.DataFrame(unlog_pred_price, columns=["unlog_pred_price"])


  df_logs = pd.concat([log_price, log_price_pred, unlog_price, unlog_pred_price], 
                       axis=1)
  
  return df_logs